# Import libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

import pyspark.sql.functions as f

## Instantiate a Spark Session

In [2]:
spark = SparkSession \
    .builder \
    .appName("Processing incident event log data") \
    .getOrCreate()

## Read in the data set

In [3]:
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("mode", "DROPMALFORMED") \
    .load("incident_event_log.csv")

In [4]:
df.printSchema()

root
 |-- number: string (nullable = true)
 |-- incident_state: string (nullable = true)
 |-- active: string (nullable = true)
 |-- reassignment_count: string (nullable = true)
 |-- reopen_count: string (nullable = true)
 |-- sys_mod_count: string (nullable = true)
 |-- made_sla: string (nullable = true)
 |-- caller_id: string (nullable = true)
 |-- opened_by: string (nullable = true)
 |-- opened_at: string (nullable = true)
 |-- sys_created_by: string (nullable = true)
 |-- sys_created_at: string (nullable = true)
 |-- sys_updated_by: string (nullable = true)
 |-- sys_updated_at: string (nullable = true)
 |-- contact_type: string (nullable = true)
 |-- location: string (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- u_symptom: string (nullable = true)
 |-- cmdb_ci: string (nullable = true)
 |-- impact: string (nullable = true)
 |-- urgency: string (nullable = true)
 |-- priority: string (nullable = true)
 |-- assignment_group: 

## Filter new incidents that exceeded the target SLA 

In [5]:
exceeded_sla = df.filter((f.col('incident_state') == 'New') & (f.col('made_sla') == True)).show(vertical=True)
exceeded_sla

-RECORD 0----------------------------------
 number                  | INC0000045      
 incident_state          | New             
 active                  | true            
 reassignment_count      | 0               
 reopen_count            | 0               
 sys_mod_count           | 0               
 made_sla                | true            
 caller_id               | Caller 2403     
 opened_by               | Opened by  8    
 opened_at               | 29/2/2016 01:16 
 sys_created_by          | Created by 6    
 sys_created_at          | 29/2/2016 01:23 
 sys_updated_by          | Updated by 21   
 sys_updated_at          | 29/2/2016 01:23 
 contact_type            | Phone           
 location                | Location 143    
 category                | Category 55     
 subcategory             | Subcategory 170 
 u_symptom               | Symptom 72      
 cmdb_ci                 | ?               
 impact                  | 2 - Medium      
 urgency                 | 2 - M

In [6]:
## Number of records of which incidents exceeded SLA and had an incident state value of New

exceeded_sla = df.filter((f.col('incident_state') == 'New') & (f.col('made_sla') == True)).count()
exceeded_sla

36407

## Filter number of active cases 

In [7]:
df.filter(df.active == True) \
    .select('number', 'active') \
    .dropDuplicates() \
    .count()

24918

## Filter out highly urgent incidents that are yet to be resolved

In [8]:
urgent_and_active = df.filter((f.col('active') == True) & (f.col('urgency') == '1 - High')).show(vertical=True)
urgent_and_active

-RECORD 0-------------------------------------
 number                  | INC0000261         
 incident_state          | Active             
 active                  | true               
 reassignment_count      | 0                  
 reopen_count            | 0                  
 sys_mod_count           | 0                  
 made_sla                | true               
 caller_id               | Caller 1393        
 opened_by               | Opened by  128     
 opened_at               | 29/2/2016 12:54    
 sys_created_by          | Created by 67      
 sys_created_at          | 29/2/2016 12:58    
 sys_updated_by          | Updated by 300     
 sys_updated_at          | 29/2/2016 12:58    
 contact_type            | Phone              
 location                | Location 197       
 category                | Category 42        
 subcategory             | Subcategory 31     
 u_symptom               | Symptom 273        
 cmdb_ci                 | ?                  
 impact      

## Filter number of resolved incidents from the most logged incidents

In [31]:
df.filter(df.incident_state == 'Resolved') \
    .select('number') \
    .groupBy('number') \
    .agg({'number': 'count'}) \
    .withColumnRenamed('count(number)', 'incident_count') \
    .sort(desc('incident_count')) \
    .show(vertical=True)

-RECORD 0--------------------
 number         | INC0011206 
 incident_count | 12         
-RECORD 1--------------------
 number         | INC0007229 
 incident_count | 12         
-RECORD 2--------------------
 number         | INC0019396 
 incident_count | 9          
-RECORD 3--------------------
 number         | INC0044260 
 incident_count | 7          
-RECORD 4--------------------
 number         | INC0007593 
 incident_count | 7          
-RECORD 5--------------------
 number         | INC0020788 
 incident_count | 7          
-RECORD 6--------------------
 number         | INC0003419 
 incident_count | 6          
-RECORD 7--------------------
 number         | INC0004822 
 incident_count | 6          
-RECORD 8--------------------
 number         | INC0033952 
 incident_count | 5          
-RECORD 9--------------------
 number         | INC0001760 
 incident_count | 5          
-RECORD 10-------------------
 number         | INC0001471 
 incident_count | 5          
-RECORD 11

## Impact vs Urgency - Is there any correlation between the urgency of an incident and it's impact?

In [29]:
impact_vs_urgency = df.select((f.col('urgency') == '3 - Low') & (f.col('impact') == '1 - High')).show(vertical=True)

## From the output, there is clearly no direct correlation between the level of urgency of an 
## incident and the impact the incident had.
## The fact that an incident was classified as highly urgent does not necessarily imply that 
## the incident(resolved or not) made significant impact. 
## A possible explanation could be that an incident was classified as high in urgency, even though 
## it wasn't that much significant to operations.
## Tweaking the values for the 'urgency' and 'impact' attributes proves this. 

-RECORD 0----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 1----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 2----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 3----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 4----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 5----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 6----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 7----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) | false 
-RECORD 8----------------------------------------------
 ((urgency = 3 - Low) AND (impact = 1 - High)) |

## Number of incidents resolved using a Knowledge Base

In [40]:
df.filter((f.col('incident_state') == 'Resolved') & (f.col('knowledge') == True)).count()

3715

In [41]:
df.filter((f.col('incident_state') == 'Resolved') & (f.col('knowledge') == False)).count()


### From the abov, fewer incidents were resolved using a Knowledge base. 
### However, they are significantly many to justify keeping a knowledge base.

22036